## 导入需要用到的包

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor as XGBR
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from mpl_toolkits.mplot3d import Axes3D 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import log_loss
from sklearn.decomposition import PCA
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import numpy as np
import pandas as pd
import os

## 数据读取

#### logP

In [ ]:
X_ = pd.read_excel('./S1.xlsx', sheet_name='S1',index_col=0).iloc[:,:]
logP = pd.read_excel('./S1.xlsx', sheet_name='logP')
Y = logP.iloc[:,1]
X = X_.loc[logP.iloc[:,0],:]
display(X)

#### celluptake1

In [ ]:
X_ = pd.read_excel('./S1.xlsx', sheet_name='S1',index_col=0).iloc[:,:]
logP = pd.read_excel('./S1.xlsx', sheet_name='celluptake1')
Y = logP.iloc[:,1]
X = X_.loc[logP.iloc[:,0],:]
display(X)

## 预打分

In [ ]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,test_size=0.2,random_state=100)
rfc = RandomForestRegressor(random_state=60)
CV_score = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
regressor = rfc.fit(Xtrain, Ytrain)
score_test = regressor.score(Xtest,Ytest)
print("5cv:",CV_score,"TEST:",score_test)

### 设置train_test_split的随机数种子

In [ ]:
# CV = 0
# test =0
for i in range(500):
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,test_size=0.2,random_state=i)
    rfc = RandomForestRegressor(random_state=60)
    CV_score = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    regressor = rfc.fit(Xtrain, Ytrain)
    score_test = regressor.score(Xtest,Ytest)
#     if CV < CV_score and test < score_test:
#         CV = CV_score
#         test = score_test
    if CV_score>0.65 and score_test>0.65:
        print("5cv:",CV_score,"TEST:",score_test,"random_state:",i)

### 选择分数较高的train_test_split中的random_state

In [ ]:
# Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,test_size=0.2,random_state=205)# logP
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,test_size=0.2,random_state=408)# celluptake1
rfc = RandomForestRegressor(random_state=60)
CV_score = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
regressor = rfc.fit(Xtrain, Ytrain)
score_test = regressor.score(Xtest,Ytest)
print("5cv:",CV_score,"TEST:",score_test)

### 划分数据集后对应粒径
#### （不运行，之前已保存）

In [ ]:
Xtrain.index

In [ ]:
logP = pd.read_excel('./S1.xlsx', sheet_name='logP',index_col=0)
logP

In [ ]:
size = logP.loc[Xtrain.index,:]
size

In [ ]:
# pd.DataFrame(size).to_excel('./logP_size.xlsx')

## RF模型调参

In [ ]:
score_5cv_all = []
for i in range(0, 200, 1):
    rfc =RandomForestRegressor(random_state=i)
    score_5cv =cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass

score_max_5cv = max(score_5cv_all)
random_state_5cv = range(0, 200)[score_5cv_all.index(max(score_5cv_all))]

print("Best_5cv score：{}".format(score_max_5cv),
      "random_5cv:{}".format(random_state_5cv))

score_5cv_all = []
for i in range(1, 400, 1):
    rfc = RandomForestRegressor(n_estimators=i
                                , random_state=random_state_5cv)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass

score_max_5cv = max(score_5cv_all)
n_est_5cv = range(1,400)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "n_est_5cv:{}".format(n_est_5cv))

score_5cv_all = []
for i in range(1, 300, 1):
    rfc = RandomForestRegressor(n_estimators=n_est_5cv
                                ,random_state=random_state_5cv
                                ,max_depth=i)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass

score_max_5cv = max(score_5cv_all)
max_depth_5cv = range(1,300)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_depth_5cv:{}".format(max_depth_5cv))

score_5cv_all = []
for i in range(1,X.shape[1]+1):
    rfc = RandomForestRegressor(n_estimators=n_est_5cv
                                ,random_state=random_state_5cv
                                ,max_depth=max_depth_5cv
                                ,max_features=i)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass

score_max_5cv = max(score_5cv_all)
max_features_5cv = range(1,X.shape[1]+1)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_features_5cv:{}".format(max_features_5cv))

### 使用调整过后的参数

In [ ]:
from sklearn.metrics import mean_absolute_error

rfc = RandomForestRegressor(random_state=21
                            ,n_estimators=51
                            ,max_depth=8
                            ,max_features=37
                           )

CV_score = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
CV_predictions = cross_val_predict(rfc, Xtrain, Ytrain, cv=5)
rmse_5cv = np.sqrt(mean_squared_error(Ytrain,CV_predictions))
mae_5cv = mean_absolute_error(Ytrain, CV_predictions)
print("r2_5cv:",CV_score,"rmse_5CV",rmse_5cv, "mae_5cv", mae_5cv)

regressor = rfc.fit(Xtrain, Ytrain)
test_predictions = regressor.predict(Xtest)
score_test = regressor.score(Xtest,Ytest)
rmse_test = np.sqrt(mean_squared_error(Ytest,test_predictions))
mae_test = mean_absolute_error(Ytest, test_predictions)
print("r2_test:",score_test,"rmse_test",rmse_test, "mae_test", mae_test)

expvspred_5cv = pd.DataFrame({'Exp_5cv': Ytrain, 'Pred_5cv':CV_predictions})
expvspred_test = pd.DataFrame({'Exp_test':Ytest,'Pred_test':test_predictions})

writer = pd.ExcelWriter('cytotoxicity_predict_RF.xlsx')
expvspred_5cv.to_excel(excel_writer=writer,sheet_name="5cv")
expvspred_test.to_excel(excel_writer=writer,sheet_name="test")
writer.save()


In [ ]:
def result_figure(y_test_train, pred_train, y_test_test, pred_test, x_label, y_label, data_neme, model_name, path):
#     os.makedirs("{}/figure".format(path), exist_ok=True)
#     os.makedirs("{}/csv".format(path), exist_ok=True)
    
    r2_train = r2_score(y_test_train, pred_train)
    r2_test = r2_score(y_test_test, pred_test)

    mse_train = MSE(y_test_train, pred_train)
    mse_test = MSE(y_test_test, pred_test)

    mae_train = MAE(y_test_train, pred_train)
    mae_test = MAE(y_test_test, pred_test)
    
#     pd.concat([pd.DataFrame(pred_train,columns=["prediction"])
#                ,pd.DataFrame(y_test_train,columns=['true'])
#                ,pd.DataFrame([r2_train],columns=["r^2"])
#                ,pd.DataFrame([np.sqrt(mse_train)],columns=['RMSE'])
#                ,pd.DataFrame([mae_train],columns=['MAE'])],axis =1).to_csv("{}/csv/{}_{}_kf.csv".format(path, data_neme, model_name), index=None)
    
#     pd.concat([pd.DataFrame(pred_test,columns=["prediction"])
#                ,pd.DataFrame(y_test_test,columns=['true'])
#                ,pd.DataFrame([r2_test],columns=["r^2"])
#                ,pd.DataFrame([np.sqrt(mse_test)],columns=['RMSE'])
#                ,pd.DataFrame([mae_test],columns=['MAE'])],axis =1).to_csv("{}/csv/{}_{}_test.csv".format(path, data_neme, model_name), index=None)
    
    plt.figure(figsize=[20,20], dpi=300)
    # plt.grid()
    matplotlib.rcParams['font.family'] = "Arial"
    plt.title('{}_{}'.format(data_neme, model_name),fontsize=70
              ,verticalalignment= 'baseline'
              ,horizontalalignment='center')

    xmax = int(max(list(pred_train)+list(y_test_train)+list(pred_test)+list(y_test_test)))+1
    xmin = int(min(list(pred_train)+list(y_test_train)+list(pred_test)+list(y_test_test)))-1

    plt.scatter(y_test_train, pred_train,c="royalblue", s=800 ,edgecolor="k",alpha=0.7, label="Train Set")
    plt.scatter(y_test_test, pred_test,c="tomato", s=800 ,edgecolor="k",alpha=0.7, label="Test Set")

#     line1 = plt.scatter(-2, -2,c="royalblue", s=1200 ,edgecolor="k",alpha=0.9)
#     line2 = plt.scatter(-2, -2,c="tomato", s=1200 ,edgecolor="k",alpha=0.9)

    plt.plot(range(xmin,xmax+1),range(xmin,xmax+1),c='k',linewidth=3, linestyle="--", alpha=0.6)
    plt.xlabel(x_label,fontsize=50)
    plt.ylabel(y_label,fontsize=50, fontdict={"family":"Arial"})
#     labels = [""]+[str(i) for i in range(xmin+1,xmax+1)]
    plt.yticks(ticks=range(xmin,xmax+1,2), labels=range(xmin,xmax+1,2), fontsize=40)
    plt.xticks(ticks=range(xmin,xmax+1,2), labels=range(xmin,xmax+1,2), fontsize=40)
    plt.xlim(xmin,xmax)
    plt.ylim(xmin,xmax)
    plt.legend(fontsize=50, labels=['Train Set','Test Set'], framealpha=0)
    # 标注R2和RMSE
    text = "R2_5cv={}\nRMSE_5cv={}\nR2_test={}\nRMSE_test={}\n".format(round(r2_train,2),round(np.sqrt(mse_train),2),round(r2_test,2),round(np.sqrt(mse_test),2))
#     superscript = str.maketrans("2", "²")
#     superscript_text = text.translate(superscript)
    plt.text(14, -1, text, fontsize=50, color="black")
    plt.savefig("{}_{}.png".format(data_neme, model_name))
    plt.show()
result_figure(Ytrain, CV_predictions, Ytest, test_predictions, "Actual Values", "Predicted Values", "celluptake", "RF", "./")

## 特征重要性分析

In [ ]:
importances = rfc.feature_importances_

plt.figure(figsize=[12,9])
plt.title("celluptake_Top10 importances", fontsize=40)
plt.bar(range(10), np.sort(importances)[::-1][:10],color='green')
importances_index_reverse = np.argsort(importances)[::-1][:10]
plt.xticks(range(10), X.columns[importances_index_reverse],fontsize=20,rotation=45)
plt.yticks(fontsize=20)
plt.savefig('./ct_Top10 importances.png')
plt.show()

In [ ]:
pd.DataFrame(np.sort(importances)[::-1][:10],X.columns[importances_index_reverse])

## 预测新数据

In [ ]:
generate = pd.read_csv('./generate_out_xa.csv',index_col=0)#.iloc[:,1:]
generate_prediction = regressor.predict(generate)
pd.DataFrame(generate_prediction).to_excel('./logP_pred_gene_RF.xlsx')

## 主成分分析

In [ ]:
database = pd.read_csv('./SupplementaryData_AuAgPtPd_xa.csv',index_col=0).iloc[:,1:]
generate = pd.read_csv('./generate_out_xa.csv',index_col=0)#.iloc[:,1:]

pca = PCA(n_components=3,copy=True,random_state=8)

databasePCA = pca.fit_transform(database)
pd.DataFrame(databasePCA).to_excel('./PCA_database.xlsx')
generatePCA = pca.fit_transform(generate)
pd.DataFrame(generatePCA).to_excel('./PCA_generate.xlsx')

********************

# XGBoost

### 预打分

In [ ]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,test_size=0.2,random_state=100)
XGB = XGBR(random_state=60)
CV_score = cross_val_score(XGB, Xtrain, Ytrain, cv=5).mean()
regressor = XGB.fit(Xtrain, Ytrain)
score_test = regressor.score(Xtest,Ytest)
print("5cv:",CV_score,"TEST:",score_test)

### 设置train_test_split的随机数种子，输出CV_score>0.60 and score_test>0.70的打分和random_state值
#### 不运行，已有运行结果

In [ ]:
# CV = 0
# test =0
for i in range(500):
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,test_size=0.2,random_state=i)
    XGB = XGBR(random_state=60)
    CV_score = cross_val_score(XGB, Xtrain, Ytrain, cv=5).mean()
    regressor = XGB.fit(Xtrain, Ytrain)
    score_test = regressor.score(Xtest,Ytest)
#     if CV < CV_score and test < score_test:
#         CV = CV_score
#         test = score_test
    if CV_score>0.55 and score_test>0.65:
        print("5cv:",CV_score,"TEST:",score_test,"random_state:",i)

In [ ]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,test_size=0.2,random_state=113)

In [ ]:
XGB = XGBR(random_state=60)
CV_score = cross_val_score(XGB, Xtrain, Ytrain, cv=5).mean()
regressor = XGB.fit(Xtrain, Ytrain)
score_test = regressor.score(Xtest,Ytest)
print("5cv:",CV_score,"TEST:",score_test)

### Random_state

In [ ]:
score_5cv_all = []
for i in range(0, 200, 1):
    XGB =XGBR(random_state=i)
    score_5cv =cross_val_score(XGB, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
random_state_5cv = range(0, 200)[score_5cv_all.index(max(score_5cv_all))]

print("Best_5cv score：{}".format(score_max_5cv),
      "random_state_5cv:{}".format(random_state_5cv))

### learning_rate

In [ ]:
score_5cv_all = []
for i in np.arange(0.01, 0.5, 0.01):
    XGB = XGBR(learning_rate=i
              ,random_state=random_state_5cv)
    score_5cv = cross_val_score(XGB, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
n_lr_5cv = np.arange(0.01,0.5,0.01)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "n_lr_5cv:{}".format(n_lr_5cv))

### N_estimators

In [ ]:
score_5cv_all = []
for i in range(1, 400, 1):
    XGB = XGBR(n_estimators=i
               ,learning_rate=n_lr_5cv
               ,random_state=random_state_5cv)
    score_5cv = cross_val_score(XGB, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
n_est_5cv = range(1,400)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "n_est_5cv:{}".format(n_est_5cv))

### Max_depth

In [ ]:
score_5cv_all = []
for i in range(1, 300, 1):
    XGB = XGBR(n_estimators=n_est_5cv
               ,learning_rate=n_lr_5cv
               ,random_state=random_state_5cv
               ,max_depth=i)
    score_5cv = cross_val_score(XGB, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
max_depth_5cv = range(1,300)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_depth_5cv:{}".format(max_depth_5cv))

### Gamma

In [ ]:
score_5cv_all = []
for i in np.arange(0,5,0.05):
    XGB = XGBR(n_estimators=n_est_5cv
               ,learning_rate=n_lr_5cv
               ,random_state=random_state_5cv
               ,max_depth=max_depth_5cv
               ,gamma=i)
    score_5cv = cross_val_score(XGB, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
max_gamma_5cv =  np.arange(0,5,0.05)[score_5cv_all.index(score_max_5cv)]
print("Best_5cv score：{}".format(score_max_5cv),
      "max_gamma_5cv:{}".format(max_gamma_5cv))

### Alpha

In [ ]:
score_5cv_all = []
for i in np.arange(0,5,0.05):
    XGB = XGBR(n_estimators=n_est_5cv,
               learning_rate=n_lr_5cv
               ,random_state=random_state_5cv
               ,max_depth=max_depth_5cv
               ,gamma=max_gamma_5cv 
               ,alpha=i)
    score_5cv = cross_val_score(XGB, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
max_alpha_5cv =  np.arange(0,5,0.05)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_alpha_5cv:{}".format(max_alpha_5cv))

## 使用调整过后的参数

In [ ]:
XGB = XGBR(learning_rate=n_lr_5cv
           ,n_estimators=n_est_5cv
           ,random_state=random_state_5cv
           ,max_depth=max_depth_5cv
           ,gamma =max_gamma_5cv
           ,alpha = max_alpha_5cv)
CV_score = cross_val_score(XGB, Xtrain, Ytrain, cv=5).mean()
CV_predictions = cross_val_predict(XGB, Xtrain, Ytrain, cv=5)
rmse = np.sqrt(mean_squared_error(Ytrain,CV_predictions))
print("r2_5cv:",CV_score,"rmse_5CV",rmse)
# expvspred_5cv = {'Exp': Ytrain, 'Pred':CV_predictions}
# pd.DataFrame(expvspred_5cv).to_excel('./XGB_5fcv_predictions_logP.xlsx')

In [ ]:
XGB = XGBR(learning_rate=n_lr_5cv
           ,n_estimators=n_est_5cv
           ,random_state=random_state_5cv
           ,max_depth=max_depth_5cv
           ,gamma =max_gamma_5cv
           ,alpha = max_alpha_5cv)
regressor = XGB.fit(Xtrain, Ytrain)
test_predictions = regressor.predict(Xtest)
score_test = regressor.score(Xtest,Ytest)
rmse = np.sqrt(mean_squared_error(Ytest,test_predictions))
print("r2_test:",score_test,"rmse_test",rmse)
# expvspred_test = {'Exp':Ytest,'Pred':test_predictions}
# pd.DataFrame(expvspred_test).to_excel('./XGB_test_predictions_logPsx')

In [ ]:
result_figure(Ytrain, CV_predictions, Ytest, test_predictions, "Actual Values", "Predicted Values", "Presentation_data", "XGBoost", "./results_xgb")

# KNN

### 预打分

In [ ]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,test_size=0.2,random_state=100)
knr = KNR()# 默认参数，创建空分类器
CV_score = cross_val_score(knr, Xtrain, Ytrain, cv=5).mean()
regressor = knr.fit(Xtrain, Ytrain)
score_test = regressor.score(Xtest,Ytest)
print("5cv:",CV_score,"TEST:",score_test)

### 设置train_test_split的随机数种子，输出CV_score>0.60 and score_test>0.70的打分和random_state值
#### 不运行，已有运行结果

In [ ]:
# CV = 0
# test =0
for i in range(500):
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,test_size=0.2,random_state=i)
    knr = KNR()
    CV_score = cross_val_score(knr, Xtrain, Ytrain, cv=5).mean()
    regressor = knr.fit(Xtrain, Ytrain)
    score_test = regressor.score(Xtest,Ytest)
#     if CV < CV_score and test < score_test:
#         CV = CV_score
#         test = score_test
    if CV_score>0.60 and score_test>0.70:
        print("5cv:",CV_score,"TEST:",score_test,"random_state:",i)

In [ ]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,test_size=0.2,random_state=266)

In [ ]:
knr = KNR()# 默认参数，创建空分类器
CV_score = cross_val_score(knr, Xtrain, Ytrain, cv=5).mean()
regressor = knr.fit(Xtrain, Ytrain)
score_test = regressor.score(Xtest,Ytest)
print("5cv:",CV_score,"TEST:",score_test)

### 网格搜索

In [ ]:
param_grid = [
    {  # 需遍历10次
        'weights': ['uniform'], # 参数取值范围
        'n_neighbors': [i for i in range(1, 11)]  # 使用其他方式如np.arange()也可以
        # 这里没有p参数
    },
    {  # 需遍历50次
        'weights': ['distance'],
        'n_neighbors': [i for i in range(1, 11)],
        'p': [i for i in range(1, 6)]
    }
]# 共需遍历60次

In [ ]:
from sklearn.model_selection import GridSearchCV  # CV，使用交叉验证方式获得模型正确率
grid_search = GridSearchCV(knr, param_grid, cv=5)  # 网格搜索参数

grid_search.fit(Xtrain, Ytrain)  # 网格搜索训练模型

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.score(Xtest, Ytest)

In [ ]:
knr=grid_search.best_estimator_

CV_score = cross_val_score(knr, Xtrain, Ytrain, cv=5).mean()
CV_predictions = cross_val_predict(knr, Xtrain, Ytrain, cv=5)
rmse_5cv = np.sqrt(mean_squared_error(Ytrain,CV_predictions))
regressor=knr.fit(Xtrain, Ytrain)
score_test = regressor.score(Xtest,Ytest)
test_predictions = regressor.predict(Xtest)
rmse_test = np.sqrt(mean_squared_error(Ytest,test_predictions))

print("5cv:",CV_score,"rmse_5cv:",rmse_5cv)
print("test:",score_test,"rmse_test:",rmse_test)

******

## 用全部的数据做五折

#### logP

In [ ]:
# data=pd.read_excel(r'd:\Desktop\task0\predict_model\model2\S12.xlsx',sheet_name='logP').to_numpy()

# X = data[:,3:129]
# y = data[:,2]

#### celluptake1

In [ ]:
data=pd.read_excel(r'd:\Desktop\task0\predict_model\model2\S12.xlsx',sheet_name='celluptake1').to_numpy()

X = data[:,3:129]
y = data[:,2]

## RF

In [ ]:
for i in range(0,500):
    kf = KFold(n_splits= 5,shuffle=True,random_state=i)

    score_all=[]
    for train_index, test_index in kf.split(X):
        rfr = RandomForestRegressor(random_state=60)
        # train_index 就是分类的训练集的下标，test_index 就是分配的验证集的下标
        train_x, train_y = X[train_index], y[train_index]  # 本组训练集
        test_x, test_y = X[test_index], y[test_index]  # 本组验证集
        prediction = rfr.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)

    score_mean = np.mean(score_all)
    if score_mean > 0.70:
        print('r2_score:',score_mean,"random_state:",i)

In [ ]:
kf = KFold(n_splits= 5,shuffle=True,random_state=100)

score_all=[]
for train_index, test_index in kf.split(X):
    rfr = RandomForestRegressor(random_state=60)
    train_x, train_y = X[train_index], y[train_index]
    test_x, test_y = X[test_index], y[test_index]
    prediction = rfr.fit(train_x, train_y).predict(test_x)
    score = r2_score(test_y, prediction)
    score_all.append(score)

score_mean = np.mean(score_all)
print('r2_score:',score_mean)

### random_state

In [ ]:
score_5cv_all = []
for i in range(0, 200, 1):
    score_all=[]
    for train_index, test_index in kf.split(X):
        rfr =RandomForestRegressor(random_state=i)
        train_x, train_y = X[train_index], y[train_index]
        test_x, test_y = X[test_index], y[test_index]
        prediction = rfr.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)
        
    score_mean = np.mean(score_all)
    score_5cv_all.append(score_mean)
    
score_max_5cv = max(score_5cv_all)
random_state_5cv = range(0, 200)[score_5cv_all.index(max(score_5cv_all))]

print("Best_5cv score：{}".format(score_max_5cv),
      "random_5cv:{}".format(random_state_5cv))

### n_estimators

In [ ]:
score_5cv_all = []
for i in range(1, 400, 1):
    score_all=[]
    for train_index, test_index in kf.split(X):
        rfr = RandomForestRegressor(n_estimators=i
                                   ,random_state=random_state_5cv)
        train_x, train_y = X[train_index], y[train_index]
        test_x, test_y = X[test_index], y[test_index]
        prediction = rfr.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)
        
    score_mean = np.mean(score_all)
    score_5cv_all.append(score_mean)
    
score_max_5cv = max(score_5cv_all)
n_est_5cv = range(1,400)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "n_est_5cv:{}".format(n_est_5cv))

### max_depth

In [ ]:
score_5cv_all = []
for i in range(1, 300, 1):
    score_all=[]
    for train_index, test_index in kf.split(X):
        rfr = RandomForestRegressor(n_estimators=n_est_5cv
                                ,random_state=random_state_5cv
                                ,max_depth=i)
        train_x, train_y = X[train_index], y[train_index]
        test_x, test_y = X[test_index], y[test_index]
        prediction = rfr.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)
        
    score_mean = np.mean(score_all)
    score_5cv_all.append(score_mean)
    
score_max_5cv = max(score_5cv_all)
max_depth_5cv = range(1,300)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_depth_5cv:{}".format(max_depth_5cv))

### max_features

In [ ]:
score_5cv_all = []
for i in range(int(X.shape[1]**0.1)+1, X.shape[1]):
    score_all=[]
    for train_index, test_index in kf.split(X):
        rfr = RandomForestRegressor(n_estimators=n_est_5cv
                                ,random_state=random_state_5cv
                                ,max_depth=max_depth_5cv
                                ,max_features=i)
        train_x, train_y = X[train_index], y[train_index]
        test_x, test_y = X[test_index], y[test_index]
        prediction = rfr.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)
        
    score_mean = np.mean(score_all)
    score_5cv_all.append(score_mean)
    
score_max_5cv = max(score_5cv_all)
max_features_5cv = range(int(X.shape[1]**0.1)+1, X.shape[1])[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_features_5cv:{}".format(max_features_5cv))

### 使用调整过后的参数

In [ ]:
kf = KFold(n_splits= 5,shuffle=True,random_state=100)

score_all=[]
score_all=[]
for train_index, test_index in kf.split(X):
    rfr = RandomForestRegressor(n_estimators=n_est_5cv
                            ,random_state=random_state_5cv
                            ,max_depth=max_depth_5cv
                            ,max_features=max_features_5cv)
    train_x, train_y = X[train_index], y[train_index]
    test_x, test_y = X[test_index], y[test_index]
    regressor = rfr.fit(train_x, train_y)
    prediction = regressor.predict(test_x)
    score = r2_score(test_y, prediction)
    score_all.append(score)
        
score_mean = np.mean(score_all)
print('r2_score:',score_mean)

In [ ]:
# generate = pd.read_csv('./generate.csv',index_col=0)
# generate_prediction = regressor.predict(generate.values)
# generate_prediction

********

## XGB

In [ ]:
for i in range(0,500):
    kf = KFold(n_splits= 5,shuffle=True,random_state=i)

    score_all=[]
    for train_index, test_index in kf.split(X):
        XGB = XGBR(random_state=60)
        # train_index 就是分类的训练集的下标，test_index 就是分配的验证集的下标
        train_x, train_y = X[train_index], y[train_index]  # 本组训练集
        test_x, test_y = X[test_index], y[test_index]  # 本组验证集
        prediction = XGB.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)

    score_mean = np.mean(score_all)
    if score_mean > 0.70:
        print('r2_score:',score_mean,"random_state:",i)

In [ ]:
kf = KFold(n_splits= 5,shuffle=True,random_state=407)

score_all=[]
for train_index, test_index in kf.split(X):
    XGB = XGBR(random_state=60)
    train_x, train_y = X[train_index], y[train_index]
    test_x, test_y = X[test_index], y[test_index]
    prediction = XGB.fit(train_x, train_y).predict(test_x)
    score = r2_score(test_y, prediction)
    score_all.append(score)

score_mean = np.mean(score_all)
print('r2_score:',score_mean)

### random_state

In [ ]:
score_5cv_all = []
for i in range(0, 200, 1):
    score_all=[]
    for train_index, test_index in kf.split(X):
        XGB = XGBR(random_state=i)
        train_x, train_y = X[train_index], y[train_index]
        test_x, test_y = X[test_index], y[test_index]
        prediction = XGB.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)
        
    score_mean = np.mean(score_all)
    score_5cv_all.append(score_mean)
    
score_max_5cv = max(score_5cv_all)
random_state_5cv = range(0, 200)[score_5cv_all.index(max(score_5cv_all))]

print("Best_5cv score：{}".format(score_max_5cv),
      "random_5cv:{}".format(random_state_5cv))

### learning_rate

In [ ]:
score_5cv_all = []
for i in np.arange(0.01, 0.5, 0.01):
    score_all=[]
    for train_index, test_index in kf.split(X):
        XGB = XGBR(learning_rate=i
              ,random_state=random_state_5cv)
        train_x, train_y = X[train_index], y[train_index]
        test_x, test_y = X[test_index], y[test_index]
        prediction = XGB.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)
        
    score_mean = np.mean(score_all)
    score_5cv_all.append(score_mean)
    
score_max_5cv = max(score_5cv_all)
n_lr_5cv = np.arange(0.01,0.5,0.01)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "n_lr_5cv:{}".format(n_lr_5cv))

### n_estimators

In [ ]:
score_5cv_all = []
for i in range(1, 400, 1):
    score_all=[]
    for train_index, test_index in kf.split(X):
        XGB = XGBR(n_estimators=i
               ,learning_rate=n_lr_5cv
               ,random_state=random_state_5cv)
        train_x, train_y = X[train_index], y[train_index]
        test_x, test_y = X[test_index], y[test_index]
        prediction = XGB.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)
        
    score_mean = np.mean(score_all)
    score_5cv_all.append(score_mean)
    
score_max_5cv = max(score_5cv_all)
n_est_5cv = range(1,400)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "n_est_5cv:{}".format(n_est_5cv))

### max_depth

In [ ]:
score_5cv_all = []
for i in range(1, 300, 1):
    score_all=[]
    for train_index, test_index in kf.split(X):
        XGB = XGBR(n_estimators=n_est_5cv
               ,learning_rate=n_lr_5cv
               ,random_state=random_state_5cv
               ,max_depth=i)
        train_x, train_y = X[train_index], y[train_index]
        test_x, test_y = X[test_index], y[test_index]
        prediction = XGB.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)
        
    score_mean = np.mean(score_all)
    score_5cv_all.append(score_mean)
    
score_max_5cv = max(score_5cv_all)
max_depth_5cv = range(1,300)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_depth_5cv:{}".format(max_depth_5cv))

### gamma

In [ ]:
score_5cv_all = []
for i in np.arange(0,5,0.05):
    score_all=[]
    for train_index, test_index in kf.split(X):
        XGB = XGBR(n_estimators=n_est_5cv
               ,learning_rate=n_lr_5cv
               ,random_state=random_state_5cv
               ,max_depth=max_depth_5cv
               ,gamma=i)
        train_x, train_y = X[train_index], y[train_index]
        test_x, test_y = X[test_index], y[test_index]
        prediction = XGB.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)
        
    score_mean = np.mean(score_all)
    score_5cv_all.append(score_mean)
    
score_max_5cv = max(score_5cv_all)
max_gamma_5cv =  np.arange(0,5,0.05)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_gamma_5cv:{}".format(max_gamma_5cv))

### alpha

In [ ]:
score_5cv_all = []
for i in np.arange(0,5,0.05):
    score_all=[]
    for train_index, test_index in kf.split(X):
        XGB = XGBR(n_estimators=n_est_5cv,
               learning_rate=n_lr_5cv
               ,random_state=random_state_5cv
               ,max_depth=max_depth_5cv
               ,gamma=max_gamma_5cv 
               ,alpha=i
                  )
        train_x, train_y = X[train_index], y[train_index]
        test_x, test_y = X[test_index], y[test_index]
        prediction = XGB.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)
        
    score_mean = np.mean(score_all)
    score_5cv_all.append(score_mean)
    
score_max_5cv = max(score_5cv_all)
max_alpha_5cv =  np.arange(0,5,0.05)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_alpha_5cv:{}".format(max_alpha_5cv))

### 使用调整过后的参数

In [ ]:
kf = KFold(n_splits= 5,shuffle=True,random_state=407)

score_all=[]
for train_index, test_index in kf.split(X):
    XGB = XGBR(n_estimators=n_est_5cv,
           learning_rate=n_lr_5cv
           ,random_state=random_state_5cv
           ,max_depth=max_depth_5cv
           ,gamma=max_gamma_5cv 
           ,alpha=max_alpha_5cv
              )
    train_x, train_y = X[train_index], y[train_index]
    test_x, test_y = X[test_index], y[test_index]
    regressor = XGB.fit(train_x, train_y)
    prediction = regressor.predict(test_x)
    score = r2_score(test_y, prediction)
    score_all.append(score)

score_mean = np.mean(score_all)
print('r2_score:',score_mean)

In [ ]:
# generate_prediction = regressor.predict(generate.values)
# generate_prediction

*********

## KNN

In [ ]:
for i in range(0,500):
    kf = KFold(n_splits= 5,shuffle=True,random_state=i)

    score_all=[]
    for train_index, test_index in kf.split(X):
        knr = KNR()
        # train_index 就是分类的训练集的下标，test_index 就是分配的验证集的下标
        train_x, train_y = X[train_index], y[train_index]  # 本组训练集
        test_x, test_y = X[test_index], y[test_index]  # 本组验证集
        prediction = knr.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)

    score_mean = np.mean(score_all)
    if score_mean > 0.75:
        print('r2_score:',score_mean,"random_state:",i)

In [ ]:
kf = KFold(n_splits= 5,shuffle=True,random_state=100)

score_all=[]
for train_index, test_index in kf.split(X):
    knr = KNR()
    train_x, train_y = X[train_index], y[train_index]
    test_x, test_y = X[test_index], y[test_index]
    prediction = knr.fit(train_x, train_y).predict(test_x)
    score = r2_score(test_y, prediction)
    score_all.append(score)

score_mean = np.mean(score_all)
print('r2_score:',score_mean)

In [ ]:
score_5cv_all = []
for i in range(1, 11):
    score_all=[]
    for train_index, test_index in kf.split(X):
        knr = KNR(weights='uniform'
                 ,n_neighbors=i)
        train_x, train_y = X[train_index], y[train_index]
        test_x, test_y = X[test_index], y[test_index]
        prediction = knr.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)
        
    score_mean = np.mean(score_all)
    score_5cv_all.append(score_mean)
    
score_max_5cv = max(score_5cv_all)
max_n_neighbors =  range(1, 11)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_n_neighbors:{}".format(max_n_neighbors))

In [ ]:
score_5cv_all = []
for i in range(1, 11):
    score_all=[]
    for train_index, test_index in kf.split(X):
        knr = KNR(weights='distance'
                 ,n_neighbors=i)
        train_x, train_y = X[train_index], y[train_index]
        test_x, test_y = X[test_index], y[test_index]
        prediction = knr.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)
        
    score_mean = np.mean(score_all)
    score_5cv_all.append(score_mean)
    
score_max_5cv = max(score_5cv_all)
max_n_neighbors =  range(1, 11)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_n_neighbors:{}".format(max_n_neighbors))

In [ ]:
score_5cv_all = []
for i in range(1, 6):
    score_all=[]
    for train_index, test_index in kf.split(X):
        knr = KNR(weights='distance'
                 ,n_neighbors=1
                 ,p=i)
        train_x, train_y = X[train_index], y[train_index]
        test_x, test_y = X[test_index], y[test_index]
        prediction = knr.fit(train_x, train_y).predict(test_x)
        score = r2_score(test_y, prediction)
        score_all.append(score)
        
    score_mean = np.mean(score_all)
    score_5cv_all.append(score_mean)
    
score_max_5cv = max(score_5cv_all)
max_p =  range(1, 6)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_p:{}".format(max_p))

### 使用调整过后的参数

In [ ]:
score_all=[]
for train_index, test_index in kf.split(X):
    knr = KNR(weights='distance'
                 ,n_neighbors=1
                 ,p=2)
    train_x, train_y = X[train_index], y[train_index]
    test_x, test_y = X[test_index], y[test_index]
    regressor = knr.fit(train_x, train_y)
    prediction = regressor.predict(test_x)
    score = r2_score(test_y, prediction)
    score_all.append(score)

score_mean = np.mean(score_all)
print('r2_score:',score_mean)

In [ ]:
# generate_prediction = regressor.predict(generate.values)
# generate_prediction